<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/dacon/titanic/titanic_option_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [재난] 타이타닉: 누가 살아남았을까?   
- 참고: [blog_Titanic 생존자 예측 옵션편](https://blog.naver.com/fkdldjs60/221901345705)
- 정확도 예측
- ML의 파라미터 변경

#### 라이브러리

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
cd drive/MyDrive/data/titanic

/content/drive/MyDrive/data/titanic


#### train, test data 불러오기

In [3]:
train = pd.read_csv('train.csv')
print(train.shape)
train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test = pd.read_csv('test.csv')
print(test.shape)
test.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


#### 문자형데이터(성별, 선착장) 숫자형 변환

In [6]:
train['Sex_encode'] = train['Sex'].replace('male', 0).replace('female', 1)
test['Sex_encode'] = train['Sex'].replace('male', 0).replace('female', 1)

In [7]:
train['Embarked_encode'] = train['Embarked'].replace('S', 0).replace('C', 1).replace('Q', 2)
test['Embarked_encode'] = test['Embarked'].replace('S', 0).replace('C', 1).replace('Q', 2)

#### 상관관계 10% 이상인 열 확인하고 리스트화

In [8]:
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_encode,Embarked_encode
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658,-0.042939,-0.030555
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307,0.543351,0.108669
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500,-0.131900,0.043835
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067,-0.093254,0.012186
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651,0.114631,-0.060606
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225,0.245489,-0.079320
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000,0.182333,0.063462
Sex_encode,-0.042939,0.543351,-0.131900,-0.093254,0.114631,0.245489,0.182333,1.000000,0.118593
Embarked_encode,-0.030555,0.108669,0.043835,0.012186,-0.060606,-0.079320,0.063462,0.118593,1.000000


In [9]:
numeric = ['Pclass', 'Fare', 'Sex_encode', 'Embarked_encode']
numeric

['Pclass', 'Fare', 'Sex_encode', 'Embarked_encode']

#### 의사결정나무 알고리즘

In [10]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [11]:
x_train = train[numeric]
x_test = test[numeric]
y_train = train['Survived']

#### 결측값 처리

In [12]:
x_train['Embarked_encode'] = x_train['Embarked_encode'].fillna(0)
x_test['Fare'] = x_test['Fare'].fillna(x_test['Fare'].mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


---
## 정확도 검증
1. Hold-out 방법: train data를 약 8대2로 나눠 학습용데이터와 테스트 데이터로 사용
2. cross validation: train data를 n 등분으로 나눠 한 등분을 테스트데이터로 사용하고 나머지를 n번 반복 사용해서 정확도 확인

- 빠르게 스코어를 예측하려면 Hold-Out이 좋고 정확도를 높이려면 Cross Validation을 사용하는 것이 좋다. 




#### 정확도: cross validation 

In [15]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model, x_train, y_train, scoring = 'accuracy', cv=10).mean()
score

0.8182272159800249

---
## ML 옵션 파라미터
- ()안에 변경할 옵션 값 넣어준다.
- example) 최대 뿌리를 10개 까지 뻗고 전체 특성 중 90%만 사용하겠다.

In [17]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=10, max_features=0.9)
model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=0.9, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [18]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model, x_train, y_train, scoring = 'accuracy', cv=10).mean()
score

0.8193133583021224

- __조금 상승__

In [19]:
model.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=0.9, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [20]:
y_test = model.predict(x_test)

In [21]:
submit = pd.read_csv('sample_submission.csv')
submit['Survived'] = y_test
submit.to_csv('final_result4.csv', index = False)